In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
from tqdm import tqdm
from torch import nn
import os

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
device

device(type='cuda', index=0)

In [ ]:
# Step 1: Load the pre-trained model
model = torchvision.models.mobilenet_v2(pretrained=True).to(device)

#Step 2: Freeze the model parameters
for param in model.parameters():
    param.requires_grad = False

In [ ]:
model.classifier = torch.nn.Linear(in_features=1280, out_features=4).to(device)

In [ ]:

# Step 4: Load the images and create dataloaders
# Assuming your dataset has 3 classes and is organized into
# folders with class names:
data_transform = transforms.Compose([
    transforms.Resize(128),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),
    transforms.RandomVerticalFlip(0.1),
    transforms.RandomRotation((-0.15,0.15))
])

val_data_transform = transforms.Compose([
    transforms.Resize(128),
    #transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225]),

])


# Load the training data
train_dataset = torchvision.datasets.ImageFolder(root='train', transform=data_transform)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=512, shuffle=True,pin_memory = True)

# Load the validation data
val_dataset = torchvision.datasets.ImageFolder(root='valid', transform=val_data_transform)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=512, shuffle=False,pin_memory = True)

In [ ]:
# Step 5: Choose an optimizer and loss function
optimizer = torch.optim.Adam(model.parameters())
loss_fn = torch.nn.CrossEntropyLoss()


num_epochs=300
best_loss = float('inf')

model = model.to(device)

In [ ]:
for epoch in range(num_epochs):
    # Training
    model.train()
    train_loss, train_acc = 0.0, 0.0

    for i, (inputs, labels) in tqdm(enumerate(train_loader)):
        optimizer.zero_grad()
        outputs = model(inputs.to(device))
        loss = loss_fn(outputs, labels.to(device))
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * inputs.size(0)
        _, predictions = torch.max(outputs, 1)
        train_acc += (predictions == labels.to(device)).sum().item()

    # Validation
    model.eval()
    val_loss, val_acc = 0.0, 0.0
    with torch.no_grad():
        for inputs, labels in val_loader:
            outputs = model(inputs.to(device))
            loss = loss_fn(outputs, labels.to(device))
            val_loss += loss.item() * inputs.size(0)
            _, predictions = torch.max(outputs, 1)
            val_acc += (predictions == labels.to(device)).sum().item()

    # Calculate average losses and accuracy
    train_loss = train_loss / len(train_loader.dataset)
    train_acc = train_acc / len(train_loader.dataset)
    val_loss = val_loss / len(val_loader.dataset)
    val_acc = val_acc / len(val_loader.dataset)

    # Print training and validation results
    print(f'Epoch {epoch + 1}: train_loss = {train_loss:.4f}, train_acc = {train_acc:.4f}, val_loss = {val_loss:.4f}, val_acc = {val_acc:.4f}')

    # Save model every epoch
    torch.save(model.state_dict(), f'best_models/model_{epoch + 1}.pt')
    
    # Save best model
    if val_loss < best_loss:
        best_loss = val_loss
        torch.save(model.state_dict(), 'best_models/best_model.pt')


20it [01:55,  5.76s/it]


Epoch 1: train_loss = 1.1131, train_acc = 0.5256, val_loss = 0.9614, val_acc = 0.6068


20it [01:53,  5.68s/it]


Epoch 2: train_loss = 0.9198, train_acc = 0.6255, val_loss = 0.8483, val_acc = 0.6480


20it [01:54,  5.72s/it]


...

20it [01:47,  5.36s/it]


Epoch 54: train_loss = 0.5150, train_acc = 0.8113, val_loss = 0.5439, val_acc = 0.8006


20it [01:49,  5.46s/it]


Epoch 55: train_loss = 0.5111, train_acc = 0.8167, val_loss = 0.5505, val_acc = 0.8006


15it [01:25,  5.73s/it]